# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The api, however, provides you with a lot of information that will not be pertinent to your analysis. YOu will pull data from the api and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the api will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

In [1]:
import json
import requests
import data
import sys
import pandas as pd
import csv


In [2]:

print(sys.path)

['/Users/leratsayukova/Documents/Flatiron/Bikes_Repo', '/Users/leratsayukova/opt/anaconda3/lib/python38.zip', '/Users/leratsayukova/opt/anaconda3/lib/python3.8', '/Users/leratsayukova/opt/anaconda3/lib/python3.8/lib-dynload', '', '/Users/leratsayukova/opt/anaconda3/lib/python3.8/site-packages', '/Users/leratsayukova/opt/anaconda3/lib/python3.8/site-packages/aeosa', '/Users/leratsayukova/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/Users/leratsayukova/.ipython']


In [3]:
url= 'https://api.yelp.com/v3/businesses/search'

In [4]:
client_id = 'UPd8KVfQybexrmKSjNF-mA'
api_key = 'TQ3V7mYVhRo1vOV7HFkQEuG-QKC7eVgNjpOkfkGD43EnLnLLY2ub-owG779hx-vZI6YX6YQZSxmE9kOjoo8iQYMx3mq0pwqenQ3Jc_vYjMZCci2PPC-BevZVmZomYHYx'




___

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

- Write a function to make a call to the yelp API

In [5]:
headers = {'Authorization':'Bearer {}'.format(api_key),}
 

In [6]:

term= 'Bike Shop'
location='Austin'
categories='Bikes'

In [7]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
    
            }

In [8]:
def yelp_call(url_params, api_key):
    
    response= requests.get(url, headers=headers, params=url_params)  
    
    if response.status_code==200:
        return response.json()
     
    else: 
        return response.status_code
    


In [9]:
response= yelp_call(url_params, api_key)

## can we make a loop for the function to call itself ?

In [10]:
business_data= response['businesses']

In [11]:
response['total']

140

In [12]:
business_data[0]

{'id': 'wfKxBxJ8RFZj8jOB6Lpn-Q',
 'alias': 'bicycle-sport-shop-austin-2',
 'name': 'Trek Bicycle Lamar',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/d3F2l_l2O-idm3TMUMgWNQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bicycle-sport-shop-austin-2?adjust_creative=UPd8KVfQybexrmKSjNF-mA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=UPd8KVfQybexrmKSjNF-mA',
 'review_count': 280,
 'categories': [{'alias': 'bikes', 'title': 'Bikes'},
  {'alias': 'bikerentals', 'title': 'Bike Rentals'},
  {'alias': 'bike_repair_maintenance', 'title': 'Bike Repair/Maintenance'}],
 'rating': 4.5,
 'coordinates': {'latitude': 30.259626, 'longitude': -97.75817},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '517 S Lamar Blvd',
  'address2': '',
  'address3': '',
  'city': 'Austin',
  'zip_code': '78704',
  'country': 'US',
  'state': 'TX',
  'display_address': ['517 S Lamar Blvd', 'Austin, TX 78704']},
 'phone': '+15124773472',
 'display_phone':

In [13]:
# business_data[0].keys()

**What data do we want from each business?**

In [14]:
# business_data[0]['name']
# business_data[0]['rating']
# business_data[0]['review_count']
# business_data[0]['location']['zip_code']


    

- Write a function to parse the API response so that you can easily insert the data in to the DB

In [15]:
def parse_results(results):
    parsed_results =[]
    for biz in results:
        biz_info= ( biz['id'],
                   biz['name'],
                   biz['rating'],
                   biz['review_count'],
                   biz['location']['zip_code'])
        parsed_results.append(biz_info)
    
    return parsed_results



In [16]:
# parse_results(business_data)

In [17]:
parsed_results= parse_results(business_data)
parsed_results

[('wfKxBxJ8RFZj8jOB6Lpn-Q', 'Trek Bicycle Lamar', 4.5, 280, '78704'),
 ('WT_d47o-V5xlMNx8trI0-A', 'Monkey Wrench Bicycles', 5.0, 94, '78751'),
 ('oTujfSf88bPOcUWEmDBzjQ', 'The Peddler Bike Shop', 4.5, 176, '78751'),
 ('-Cza7JtBZZ7nuXZeStwPAA', 'Bike Farm', 4.5, 104, '78756'),
 ('1yvLoRRt15RraRMlrdUBTg', 'Cycleast', 5.0, 67, '78702'),
 ('KVPUN4yU-2juc8Pc4sxKVQ', 'Clown Dog Bikes', 5.0, 134, '78705'),
 ('4EdSNL5cShH-ZNsUbwWSJQ', 'Trek Bicycle Research', 4.5, 93, '78759'),
 ('ZtuzXaoMnY1gd0kMZFdpcw', "Mellow Johnny's Bike Shop", 4.0, 205, '78701'),
 ('lXodVpk5ZUOVymDBlb10Zg', 'Trek Bicycle Guadalupe', 4.5, 10, '78705'),
 ('2lIEXCMqbUaYJ98_cwAd2A', 'East Side Pedal Pushers', 4.5, 128, '78702'),
 ('xxFBa5ZuMb0S92wxYeAOcQ', 'Trek Bicycle Bee Cave', 4.0, 13, '78733'),
 ('JN_AiBjGmF4dDdJIYKT7SA', 'Cycle Progression', 4.5, 31, '78751'),
 ('-4SfHHiTVTLeOEt8TF0nTQ', 'Bikealot', 4.5, 30, '78745'),
 ('gWhCMZVm0ITZC7KxI3M2Pw', 'Texas Cycle Werks', 4.5, 37, '78735'),
 ('18UGjpTexL3nRJ4UrMSM7w', 'Trek

- Write a function to take your parsed data and add it to the csv file where you will store all of your results. 

In [18]:
# pr_df= pd.DataFrame(parsed_results, columns= 
#              ['name', 'rating', 'review_count', 'zipcode'])

In [19]:
# pr_csv=pr_df.to_csv(path_or_buf='/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_data')

In [20]:
def df_save(csv_filepath, parsed_results):
    
    
    pr_df= pd.DataFrame(parsed_results, columns= 
             ['id','name', 'rating', 'review_count', 'zipcode'])
    
    pr_csv=pr_df.to_csv(path_or_buf=csv_filepath, mode="a", header="False")
   
    return print("Results added!")
   

In [21]:
# df_save('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_data', parsed_results)

- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [22]:
from helpers import *

**^^Not working 

In [23]:
def yelp_data(url_params, api_key):
    yelp_call(url_params, api_key)
    parse_results(business_data)
    df_save('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_data', parsed_results)
    our_data= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_data')
    return our_data

In [24]:
# yelp_data(url_params,api_key)

In [26]:


# create a variable  to keep track of which result you are in. 
cur = 0
num= response['total']

#set up a while loop to go through and grab the result 
while cur <= num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results =  yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    new_parsed_results = parse_results(results['businesses'])
    
    # use your function to insert your parsed results into the db
    df_save('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/bikes_data', new_parsed_results)
    
    
    #increment the counter by 50 to move on to the next results
    cur += 50

Results added!
Results added!
Results added!


In [27]:
# results

{'businesses': [{'id': 'ILH9kf0X7IkiwlZqWQTUGg',
   'alias': 'flying-piston-motorcycles-round-rock',
   'name': 'Flying Piston Motorcycles',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/mSgpVf6_4--6uBR0iX6KgQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/flying-piston-motorcycles-round-rock?adjust_creative=UPd8KVfQybexrmKSjNF-mA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=UPd8KVfQybexrmKSjNF-mA',
   'review_count': 26,
   'categories': [{'alias': 'motorcyclerepair', 'title': 'Motorcycle Repair'}],
   'rating': 3.5,
   'coordinates': {'latitude': 30.5165501, 'longitude': -97.6970596},
   'transactions': [],
   'location': {'address1': '1401 Sam Bass Rd',
    'address2': None,
    'address3': '',
    'city': 'Round Rock',
    'zip_code': '78681',
    'country': 'US',
    'state': 'TX',
    'display_address': ['1401 Sam Bass Rd', 'Round Rock, TX 78681']},
   'phone': '+15123108514',
   'display_phone': '(512) 310-8514',
   'distan

In [28]:
Bikes_csv= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/bikes_data')
pd.set_option('display.max_rows', None, 'display.max_columns', None)
Bikes_csv

,Unnamed: 0,id,name,rating,review_count,zipcode
0,0.0,wfKxBxJ8RFZj8jOB6Lpn-Q,Trek Bicycle Lamar,4.5,280,78704
1,1.0,WT_d47o-V5xlMNx8trI0-A,Monkey Wrench Bicycles,5.0,94,78751
2,2.0,oTujfSf88bPOcUWEmDBzjQ,The Peddler Bike Shop,4.5,176,78751
3,3.0,-Cza7JtBZZ7nuXZeStwPAA,Bike Farm,4.5,104,78756
4,4.0,1yvLoRRt15RraRMlrdUBTg,Cycleast,5.0,67,78702
5,5.0,KVPUN4yU-2juc8Pc4sxKVQ,Clown Dog Bikes,5.0,134,78705
6,6.0,4EdSNL5cShH-ZNsUbwWSJQ,Trek Bicycle Research,4.5,93,78759
7,7.0,ZtuzXaoMnY1gd0kMZFdpcw,Mellow Johnny's Bike Shop,4.0,205,78701
8,8.0,lXodVpk5ZUOVymDBlb10Zg,Trek Bicycle Guadalupe,4.5,10,78705
9,9.0,2lIEXCMqbUaYJ98_cwAd2A,East Side Pedal Pushers,4.5,128,78702


In [ ]:
biz_ids=Bikes_csv.loc[:,'id'].to_list()
biz_ids_lst=list(set(biz_ids))


In [ ]:
biz_ids_lst

In [ ]:
biz_id_1=biz_ids[0]
biz_id_1

___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

- In order to pull the reviews, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

- Write a function that takes a business id and makes a call to the API for reivews


- Write a function to parse out the relevant information from the reviews

- Write a function to save the parse data into a csv file containing all of the reviews. 

- Combine the functions above into a single script  

In [ ]:
# url_params_reviews = {
#                 "term": term.replace(' ', '+'),
#                 "location": location.replace(' ', '+'),
#                 "categories" : categories,
#                 "limit": 50,
    
#             }

In [ ]:
biz_ids_lst.pop(13)

In [ ]:
biz_ids_lst

In [ ]:
def yelp_call_reviews(url_review, api_key):
    response= requests.get(url_reviews, headers=headers)  
    if response.status_code==200:
        return response.json()
    else: 
        return response.status_code


In [ ]:
for bid in biz_ids_lst:
    url_reviews='https://api.yelp.com/v3/businesses/' + bid + '/reviews'
    url_data = yelp_call_reviews(url_reviews, api_key)

In [ ]:
url_data = yelp_call_reviews(url_reviews, api_key)

In [ ]:
reviews

In [ ]:
url_data['reviews']

In [ ]:
review_data= yelp_call_reviews(url_data, api_key)
reviews= review_data['reviews']


In [ ]:
for review in reviews:
    print(review)

In [ ]:
biz_ids_lst

In [ ]:
reviews

In [ ]:
def yelp_reviews(url_params_reviews, api_key):
    yelp_call_reviews(biz_ids_lst, api_key)
    parse_reviews(reviews)
    df_save_reviews('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_reviews', parsed_reviews)
    our_data= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_reviews')
    return our_data

In [ ]:
def parse_reviews(bid, reviews):
    parsed_reviews =[]
    for review in reviews:
        review_info= ( bid,
                    review['id'],
                    review['text'],
                    review['rating'] 
                           )
        parsed_reviews.append(review_info)
        
    return parsed_reviews


In [ ]:
def df_save_reviews(csv_filepath, parsed_reviews):
    
    
    pr_df= pd.DataFrame(parsed_reviews, columns= 
             ['business_id','id','text', 'rating'])
    
    pr_csv=pr_df.to_csv(path_or_buf=csv_filepath, mode="a", header=False)
   
    return print("Results added!")
            

In [ ]:
for bid in biz_ids_lst[:5]:
# #     print(bid)
    url_reviews='https://api.yelp.com/v3/businesses/' + bid + '/reviews'
    url_data = yelp_call_reviews(url_reviews, api_key)
    new_reviews = yelp_call_reviews(bid, api_key)
    #after you get your results you can now use your function to parse those results
    new_parsed_reviews = parse_reviews(bid,new_reviews['reviews'])
     # use your function to insert your parsed results into the db
    df_save_reviews('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_reviews', new_parsed_reviews)
   


In [ ]:
pd.

In [ ]:
parsed_reviews= parse_reviews(reviews)

In [ ]:
set(parsed_reviews)

In [ ]:
Reviews_csv= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes/data/csv_reviews', 
            names=['business_id', 'review_id', 'text', 'rating']).reset_index(drop=True)

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


___

In [ ]:
rev_csv= Reviews_csv.head(9)
bik_csv= Bikes_csv.head(9)

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0
num = response['total']*3
#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20